<a href="https://colab.research.google.com/github/drysmakhanova/Robt403/blob/main/IKLab7_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers 
from keras.utils import np_utils 
from keras import backend as K 
import pandas as pd
from sklearn.model_selection import train_test_split
import random


In [ ]:
data = pd.read_csv("dict1.csv", header = None, names = ["Angles", "XY"])


In [ ]:
train = data['XY'].to_numpy()
labels = data['Angles'].to_numpy()

Here we use for labels translation and rotation values and keep only X,Y coordinaties and Rotation values.

In [ ]:
X = list()
Y = list()
for i in range(len(train)):
    train[i] = train[i].strip('(array([').strip(']))')
    train[i] = train[i].replace(' ', '')
    train[i] = train[i].replace('  ', '')
    train[i] = train[i].replace(',0.25]),array([0.,0.,', ',')
    train[i] = train[i].replace(',2.5000e-01]),array([0.,0.,', ',')
    train[i] = train[i].replace(',0.25]),array([0.e+00,0.e+00,', ',')
    result = [float(val) for val in train[i].split(',')]
    result.pop()
    result.pop()
    X.append(result)
    labels[i] = labels[i].strip('(').strip(')')
    result = [float(val) for val in labels[i].split(',')]
    Y.append(result)

So X values are 5  joint angles values of the robot and Y are labels: X,Y,Rotation

Shuffling lists and keeping X-Y order

In [ ]:
type(X[0])

list

In [ ]:
temp = list(zip(X, Y))
random.shuffle(temp)
X,Y = zip(*temp)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), np.asarray(Y), test_size=0.25)

In [ ]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(375, 2)
(375, 5)
(125, 2)
(125, 5)


In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential()
model.add(Dense(32, input_dim = 2, activation = 'elu'))
model.add(Dense(64, activation = 'elu'))
model.add(Dense(128, activation = 'elu'))
model.add(Dense(256, activation = 'elu'))
model.add(Dense(128, activation = 'elu'))
model.add(Dense(64, activation = 'elu'))
model.add(Dense(32, activation = 'elu'))
model.add(Dense(5, activation='elu'))
model.compile(loss=rmse, optimizer=Adam(0.001))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                96        
_________________________________________________________________
dense_17 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_18 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_19 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_20 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_21 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_22 (Dense)             (None, 32)               

In [ ]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2566
Epoch 2/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2579
Epoch 3/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2566
Epoch 4/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2559
Epoch 5/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2580
Epoch 6/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2579
Epoch 7/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2576
Epoch 8/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2587
Epoch 9/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2555
Epoch 10/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2552
Epoch 11/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2564
Epoch 12/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2575
Epoch 13/20
12/12 [==============================

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0) 
print("RMSE: %.2f" % (scores))

RMSE: 0.26


In [ ]:
print(model.predict(X_test[10].reshape(1,2)))
print(y_test[10])

[[0.0888394  0.1842444  0.26068982 0.21653081 0.2719487 ]]
[ 0.48  0.21  0.44 -0.19  0.19]


Elu and Selu show the best approximation